In [1]:
import pandas as pd
import loguru
from data.dataset import Sentinel2Dataset
from utils.utils import load_config

In [2]:
config = load_config(config_path="cfg/config.yaml")
BASE_DIR = config["DATASET"]["base_dir"]
VERSION = config['DATASET']['version']
BATCH_SIZE = config['TRAINING']['batch_size']
VAL_BS = config["TRAINING"]['val_bs']
NUM_WORKERS = config['TRAINING']['num_workers']
train_path = f"{BASE_DIR}/{VERSION}/train_path.csv"
val_path = f"{BASE_DIR}/{VERSION}/val_path.csv"
test_path = f"{BASE_DIR}/{VERSION}/test_path.csv"



In [3]:
df_train = pd.read_csv(train_path)
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

train_dataset = Sentinel2Dataset(df_path=df_train, train=False, augmentation=False)
val_dataset = Sentinel2Dataset(df_path=df_val, train=False, augmentation=False)

In [4]:
from data.loader import define_loaders

In [5]:
train_loader, val_loader = define_loaders(
        train_dataset,
        val_dataset,
        batch_size=VAL_BS,
        val_bs=VAL_BS,
        num_workers=NUM_WORKERS,
    )

In [ ]:
for data in train_loader:
    x_data, y_data = data
